# Compare Python GMAP with legacy GMAP

In this notebook we compare results from GMAP obtained with different options.

First we load required packages:

In [1]:
import sys
sys.path.append('../..')
import numpy as np
import pandas as pd
from gmapy.legacy.legacy_gmap import run_gmap

We use a database file from the unittests. This file is also the basis for the neutron standards evaluation 2017.

In [2]:
dbfile = '../../tests/testdata/data-2017-07-26.gma'

## Calculations with PPP correction

The Fortran version allowed to modes of execution. Uncertainties given in percentage in the database file could be interpreted as relative to the experimental data points or as relative to the fitted cross sections. The latter mode can be activated with the argument `correct_ppp=True`. The Fortran version had a bug in the calculation of the experimental correlation matrix, which was corrected in the Python version. However, also the Python version can be run with the bug via the specification `fix_ppp_bug=False`. Here we perform two calculations, one with the bug and another one without it. Another bug in the Fortran code was an incorrectly computed sensitivity matrix for SACS (=spectrum averaged cross section) measurements. This bug be activated by the argument `fix_sacs_jacbian=False`, but it's impact on the results based on this database file is insignificant (about 0.005%). **Finally, please note that iterating the GLS method will not converge to the maximum of the posterior distribution if PPP corrections are enabled.**

In [3]:
%%capture 
res1 = run_gmap(dbfile, dbtype='legacy', num_iter=3, correct_ppp=True,
                fix_ppp_bug=False, fix_sacs_jacobian=False, legacy_output=False)

res2 = run_gmap(dbfile, dbtype='legacy', num_iter=3, correct_ppp=True,
                fix_ppp_bug=True, fix_sacs_jacobian=True, legacy_output=False)

*Note*: The `%%capture` statement is there to discard the output of this cell, which would otherwise produce a warning about a division by zero caused by some experimental dataset.

## Calculation without PPP correction

We also include a calculation without the PPP correction (`correct_ppp=False`) in the comparison.
The bug (`fix_ppp_bug=False`) in the previous section has no impact on the result in this case. 

In [4]:
%%capture
res_noppp = run_gmap(dbfile, dbtype='legacy', num_iter=3, correct_ppp=False, legacy_output=False)

### Preparation of the comparison table

The result table as returned by `run_gmap` not only contains the estimates of the cross sections but also the estimates of the normalization factors associated with so-called shape experiments (i.e., measurements affected by an unknown scaling factor). Furthermore, it contains the fission spectrum which is, however, assumed to be perfectly known and therefore not adjusted. In order to retrieve the updated cross sections, we filter the unwanted rows from the result tables. To this end, we use the fact that the `NODE` column of all cross sections starts with `xsid`.

In [5]:
rres1 = res1['table'][res1['table'].NODE.str.match('xsid')].copy()
rres2 = res2['table'][res2['table'].NODE.str.match('xsid')].copy()
rres_noppp = res_noppp['table'][res_noppp['table'].NODE.str.match('xsid')].copy()

Now we create a comparison table with the results, including uncertainties, of all three calculations and display it.

In [6]:
relerr = np.abs(rres1.POST - rres2.POST) / np.abs(rres1.POST+1e-8)
relerr2 = np.abs(rres1.POST - rres2.POST) / rres1.POSTUNC
cmptable = pd.DataFrame.from_dict({'REAC': rres1.DESCR, 'ENERGY': rres1.ENERGY, 
                                   'ForGMAP': rres1.POST, 'ForGMAP-UNC': rres1.POSTUNC,
                                   'PyGMAP': rres2.POST, 'PyGMAP-UNC': rres2.POSTUNC,
                                   'For/PyGMAP-noppp': rres_noppp.POST, 'For/PyGMAP-noppp-UNC': rres_noppp.POSTUNC,
                                   'RELERR': relerr, 'RELERR2': relerr2})
cmptable

,REAC,ENERGY,ForGMAP,ForGMAP-UNC,PyGMAP,PyGMAP-UNC,For/PyGMAP-noppp,For/PyGMAP-noppp-UNC,RELERR,RELERR2
0,"6Li(n,a)",2.530000e-08,938.130881,0.544432,938.131177,0.544431,938.151536,0.544671,3.153103e-07,0.000543
1,"6Li(n,a)",9.400000e-06,48.639692,0.028207,48.639696,0.028207,48.640807,0.028220,8.358777e-08,0.000144
2,"6Li(n,a)",1.500000e-04,12.160778,0.004038,12.160772,0.004038,12.160907,0.004039,5.053357e-07,0.001522
3,"6Li(n,a)",2.500000e-04,9.413062,0.003676,9.413054,0.003676,9.413174,0.003677,8.632639e-07,0.002211
4,"6Li(n,a)",3.500000e-04,7.951957,0.003027,7.951949,0.003027,7.952045,0.003028,1.100589e-06,0.002891
...,...,...,...,...,...,...,...,...,...,...
958,SS-PU1,2.530000e-08,11.927023,2.574226,11.928748,2.574599,11.942983,2.583492,1.446801e-04,0.000670
959,SF-PU1,2.530000e-08,1023.918068,10.808002,1023.798550,10.806693,1022.798588,10.767231,1.167260e-04,0.011058
960,SG-PU1,2.530000e-08,361.801318,6.151239,361.794384,6.151121,361.743671,6.147610,1.916717e-05,0.001127
961,NU-PU1,2.530000e-08,2.939857,0.006445,2.939901,0.006445,2.940121,0.006447,1.486287e-05,0.006780


We may want to save this dataframe as an Excel file for further investigation:

In [10]:
try:
    cmptable.to_excel('gmap_comparison.xlsx')
except ModuleNotFoundError:
    print('`pandas` requires the `openpyxl` package to save `.xlsx` files')

`pandas` requires the `openpyxl` package to save `.xlsx` files
